In [15]:
import pandas as pd
import mysql.connector
import re
conn = mysql.connector.connect(username= "root",
                               host = "127.0.0.1",
                               password = "Ananthu@123",
                               database = "data_spark")

cursor = conn.cursor()

In [17]:
create_customer_table = """CREATE TABLE customer (
    customerkey INT PRIMARY KEY,
    gender VARCHAR(10),
    name VARCHAR(100),
    city VARCHAR(100),
    state_code VARCHAR(100),
    state VARCHAR(100),
    zip_code VARCHAR(20),
    country VARCHAR(100),
    continent VARCHAR(50),
    birthday DATE,
    age INT,
    segment VARCHAR(50)
);"""
cursor.execute(create_customer_table)
conn.commit()

In [20]:
import pandas as pd
from datetime import datetime

# Load the CSV file into a DataFrame
df_customer = pd.read_csv('Customers.csv', encoding='latin1')

# Normalize column names: make them lowercase and replace spaces with underscores
df_customer.columns = df_customer.columns.str.lower()
df_customer.columns = df_customer.columns.str.replace(" ", "_")

# Convert the 'birthday' column to datetime format
df_customer['birthday'] = pd.to_datetime(df_customer['birthday'], errors='coerce')

# Function to calculate age based on birthdate
def calculate_age(birthdate):
    if pd.isnull(birthdate):
        return None
    today = datetime.today()
    age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
    return age

# Apply the age calculation function
df_customer['age'] = df_customer['birthday'].apply(calculate_age)

# Function to categorize age into segments
def age_segment(age):
    if pd.isnull(age):
        return 'unknown'
    elif age < 25:
        return 'young adult'
    elif age < 55:
        return 'middle age'
    else:
        return 'senior citizen'

# Apply the age segmentation function
df_customer['segment'] = df_customer['age'].apply(age_segment)

# Fill NaN values in the 'state_code' column with "NA"
df_customer['state_code'] = df_customer['state_code'].fillna("NA")


In [21]:
df_customer.head()

,customerkey,gender,name,city,state_code,state,zip_code,country,continent,birthday,age,segment
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,1939-07-03,85,senior citizen
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,1979-09-27,44,middle age
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,1947-05-26,77,senior citizen
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,1957-09-17,66,senior citizen
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,1965-11-19,58,senior citizen


In [23]:
import pymysql
import pandas as pd
from datetime import datetime

# Assuming df_customer is already populated
# df_customer = pd.read_csv('Customers.csv', encoding='latin1')

# Establish connection to the database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='Ananthu@123',
    database='data_spark'
)

# Create a cursor object
cursor = conn.cursor()

# SQL query to insert or update customer data
insert_customer_values = '''
    INSERT INTO customer (
        customerkey,
        gender,
        name,
        city,
        state_code,
        state,
        zip_code,
        country,
        continent,
        birthday,
        age,
        segment
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        gender = VALUES(gender),
        name = VALUES(name),
        city = VALUES(city),
        state_code = VALUES(state_code),
        state = VALUES(state),
        zip_code = VALUES(zip_code),
        country = VALUES(country),
        continent = VALUES(continent),
        birthday = VALUES(birthday),
        age = VALUES(age),
        segment = VALUES(segment)
'''

# Ensure that birthday is in the correct datetime format for MySQL
df_customer['birthday'] = pd.to_datetime(df_customer['birthday'], errors='coerce').dt.strftime('%Y-%m-%d')

# Iterate over each row in the DataFrame and execute the INSERT statement
for index, row in df_customer.iterrows():
    # Handle potential NaN values by replacing them with None (NULL in SQL)
    cursor.execute(insert_customer_values, (
        row['customerkey'] if pd.notnull(row['customerkey']) else None,
        row['gender'] if pd.notnull(row['gender']) else None,
        row['name'] if pd.notnull(row['name']) else None,
        row['city'] if pd.notnull(row['city']) else None,
        row['state_code'] if pd.notnull(row['state_code']) else None,
        row['state'] if pd.notnull(row['state']) else None,
        row['zip_code'] if pd.notnull(row['zip_code']) else None,
        row['country'] if pd.notnull(row['country']) else None,
        row['continent'] if pd.notnull(row['continent']) else None,
        row['birthday'] if pd.notnull(row['birthday']) else None,
        row['age'] if pd.notnull(row['age']) else None,
        row['segment'] if pd.notnull(row['segment']) else None
    ))

# Commit the transaction to save the changes to the database
conn.commit()

# Close the cursor and connection to clean up resources
cursor.close()
conn.close()
